# Clean School Data

In [1]:
# Import the required libraries
import requests
import json
import pandas as pd


In [2]:

#Import Population Data
#encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1252', 'utf-16']

file_path = "Resources\dv331_schoollocations2022.csv"
schools_df = pd.read_csv(file_path, encoding="latin-1")

columns_to_drop = ["Entity_Type","SCHOOL_NO","School_Status","Address_Line_2",
                   "Address_State","Postal_Address_Line_1","Postal_Address_Line_1",
                   "Postal_State"]
#population_df = population_df.drop(columns_to_drop, axis=1)

#new_column_names = {
    #"SEX: Sex": "Gender",
    #"AGE: Age": "Age",
    #"REGION_TYPE: Geography Level":"Geography_Level",
    #"ASGS_2021: Region":"ABS_SA2_KEY",
    #"FREQ: Frequency":"Frequency",
    #"TIME_PERIOD: Time Period":"Year",
    #"OBS_VALUE": "Estimated_Resident_Population"
   # }
#population_df = population_df.rename(columns=new_column_names)
schools_df

,Gender,Age,Geography_Level,ABS_SA2_KEY,Frequency,Year,Estimated_Resident_Population
0,3: Persons,TOT: All ages,AUS: Australia,AUS: Australia,A: Annual,2001,19274701
1,3: Persons,TOT: All ages,AUS: Australia,AUS: Australia,A: Annual,2002,19495210
2,3: Persons,TOT: All ages,AUS: Australia,AUS: Australia,A: Annual,2003,19720737
3,3: Persons,TOT: All ages,AUS: Australia,AUS: Australia,A: Annual,2004,19932722
4,3: Persons,TOT: All ages,AUS: Australia,AUS: Australia,A: Annual,2005,20176844
...,...,...,...,...,...,...,...
3482068,2: Females,8599: 85 and over,GCCSA: Greater Capital City Statistical Areas,9OTER: Other Territories,A: Annual,2017,28
3482069,2: Females,8599: 85 and over,GCCSA: Greater Capital City Statistical Areas,9OTER: Other Territories,A: Annual,2018,28
3482070,2: Females,8599: 85 and over,GCCSA: Greater Capital City Statistical Areas,9OTER: Other Territories,A: Annual,2019,32
3482071,2: Females,8599: 85 and over,GCCSA: Greater Capital City Statistical Areas,9OTER: Other Territories,A: Annual,2020,36


In [3]:
#Import mapping file - use to filter to sample postcodes for further analysis
# Specify the file path and sheet name
file_path = "Resources/Melbourne Postcodes.xlsx"
sheet_name = "Mapping"

# Read the specified sheet into a DataFrame
melb_postcodes_df = pd.read_excel(file_path, sheet_name=sheet_name)
melb_postcodes_df= melb_postcodes_df.dropna(subset=['ABS_SA2_KEY'])

# Display the DataFrame
melb_postcodes_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,RURAL_TOWNSHIP,POST_CODE,ABS_SA2_KEY,HOUSE_LOCALITY,SUBURB_NAME,COMMENTS,SUBURB_POSTCODE_COMMENTS
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3053,206041117: Carlton,CARLTON,Carlton,NaN,Carlton 3053
1,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3053,206041117: Carlton,NaN,Carlton South,NaN,Carlton South
2,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3054,206071140: Carlton North - Princes Hill,CARLTON NORTH,Carlton North,Shared with City of Yarra,Carlton North 3054 (Shared with City of Yarra)
3,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3008,206041118: Docklands,NaN,Docklands,NaN,Docklands 3008
4,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3002,206041119: East Melbourne,EAST MELBOURNE,East Melbourne,NaN,East Melbourne 3002
...,...,...,...,...,...,...,...,...,...,...
1003,Western municipalities and their suburbs,City of Wyndham,Outer,NaN,3024,213051579: Manor Lakes - Quandong,WYNDHAM VALE,Wyndham Vale,NaN,Wyndham Vale 3024
1004,Western municipalities and their suburbs,City of Wyndham,Outer,Rural localities,3030,213011570: Derrimut,NaN,Cocoroc,NaN,Cocoroc 3030
1005,Western municipalities and their suburbs,City of Wyndham,Outer,Rural localities,3338,213041571: Brookfield,EYNESBURY,Eynesbury,Shared with the Shire of Melton,Eynesbury 3338 (Shared with the Shire of Melton)
1008,Western municipalities and their suburbs,City of Wyndham,Outer,Rural localities,3024,213051579: Manor Lakes - Quandong,MAMBOURIN,Mambourin,NaN,Mambourin 3024


In [4]:
# Concatenate SUBURB_NAME values for each POST_CODE
postcode_suburbs_df = melb_postcodes_df.groupby('POST_CODE')['SUBURB_NAME'].agg(lambda x: ', '.join(x)).reset_index()

# Display the resulting DataFrame
postcode_suburbs_df

,POST_CODE,SUBURB_NAME
0,3000,Melbourne
1,3002,"East Melbourne, Jolimont"
2,3003,"West Melbourne, Coode Island"
3,3006,"Southbank, South Wharf, Southbank, South Wharf"
4,3008,Docklands
...,...,...
197,3975,"Lynbrook, Lyndhurst"
198,3976,Hampton Park
199,3977,"Tooradin North, Botanic Ridge, Cranbourne, Cra..."
200,3978,"Cardinia, Lisbaun, Clyde, Clyde North"


In [5]:
population_clean_1_df = pd.merge(melb_postcodes_df, population_df, how='inner', on='ABS_SA2_KEY')
population_clean_1_df= population_clean_1_df.dropna(subset=['HOUSE_LOCALITY'])
columns_to_drop = ["RURAL_TOWNSHIP","ABS_SA2_KEY","HOUSE_LOCALITY",
                   "COMMENTS","SUBURB_NAME","SUBURB_POSTCODE_COMMENTS","Geography_Level","Frequency"]
population_clean_1_df = population_clean_1_df.drop(columns_to_drop, axis=1)
population_clean_1_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,POST_CODE,Gender,Age,Year,Estimated_Resident_Population
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2001,9529
1,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2002,10022
2,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2003,10611
3,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2004,11224
4,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2005,11924
...,...,...,...,...,...,...,...,...
957595,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2017,84
957596,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2018,97
957597,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2019,99
957598,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2020,102


In [6]:
population_clean_final_df = pd.merge(population_clean_1_df, postcode_suburbs_df, how='inner', on='POST_CODE')
new_column_names = {
    "SUBURB_NAME": "SUBURB_NAMES",
    }
population_clean_final_df = population_clean_final_df.rename(columns=new_column_names)
population_clean_final_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,POST_CODE,Gender,Age,Year,Estimated_Resident_Population,SUBURB_NAMES
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2001,9529,"Carlton, Carlton South"
1,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2002,10022,"Carlton, Carlton South"
2,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2003,10611,"Carlton, Carlton South"
3,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2004,11224,"Carlton, Carlton South"
4,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2005,11924,"Carlton, Carlton South"
...,...,...,...,...,...,...,...,...,...
489568,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2017,84,"Truganina, Weir Views, Hoppers Crossing, Laver..."
489569,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2018,97,"Truganina, Weir Views, Hoppers Crossing, Laver..."
489570,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2019,99,"Truganina, Weir Views, Hoppers Crossing, Laver..."
489571,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2020,102,"Truganina, Weir Views, Hoppers Crossing, Laver..."


In [7]:
unique_count = population_clean_final_df['POST_CODE'].nunique()

# Display the count of unique strings
print(unique_count)

197


In [8]:
# List all the unique strings in a column
unique_strings = population_clean_final_df['Age'].unique()

# Display the unique strings
print(unique_strings)

['TOT: All ages' 'A04: 0-4' 'A59: 5-9' 'A10: 10-14' 'A15: 15-19'
 'A20: 20-24' 'A25: 25-29' 'A30: 30-34' 'A35: 35-39' 'A40: 40-44'
 'A45: 45-49' 'A50: 50-54' 'A55: 55-59' 'A60: 60-64' 'A65: 65-69'
 'A70: 70-74' 'A75: 75-79' 'A80: 80-84' '8599: 85 and over']


In [9]:
# Check for NaN values in a column
has_nan = population_clean_final_df['Estimated_Resident_Population'].isna().any()

# Display the result - False (no NaN values in column), True (at least one NaN value in column)
print(has_nan)

False


In [10]:
# Export cleaned population dataset to a CSV file
population_clean_final_df.to_csv("Cleaned_Data/population_clean_final.csv", index_label="POST_CODE")
population_clean_final_df


,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,POST_CODE,Gender,Age,Year,Estimated_Resident_Population,SUBURB_NAMES
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2001,9529,"Carlton, Carlton South"
1,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2002,10022,"Carlton, Carlton South"
2,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2003,10611,"Carlton, Carlton South"
3,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2004,11224,"Carlton, Carlton South"
4,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,3: Persons,TOT: All ages,2005,11924,"Carlton, Carlton South"
...,...,...,...,...,...,...,...,...,...
489568,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2017,84,"Truganina, Weir Views, Hoppers Crossing, Laver..."
489569,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2018,97,"Truganina, Weir Views, Hoppers Crossing, Laver..."
489570,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2019,99,"Truganina, Weir Views, Hoppers Crossing, Laver..."
489571,Western municipalities and their suburbs,City of Wyndham,Outer,3029,2: Females,8599: 85 and over,2020,102,"Truganina, Weir Views, Hoppers Crossing, Laver..."
